In [1]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [3]:
import torch
import torch.optim as optim
import joblib
import optuna
from model.m27odel import MultiDecoderCondVAE
from loss.l27oss_all import integrated_loss_fn

# 1. 환경 및 데이터 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로더 및 스케일러 로드
x_scaler = joblib.load('torch/a_27_x_scaler.pkl')
x2_scaler = joblib.load('torch/s_27_x2_scaler.pkl')
x3_scaler = joblib.load('torch/s_27_x3_scaler.pkl')
c_saler = joblib.load('torch/a_27_c_scaler.pkl')
train_loader = torch.load('torch/train_loader_all.pt', weights_only=False)
val_loader = torch.load('torch/val_loader_all.pt', weights_only=False)

# 입력 차원 자동 추출 (첫 번째 배치를 통해 확인)
x_sample,x2_sample,x3_sample, c_sample = next(iter(train_loader))
x_dim = x_sample.shape[1]
x2_dim = x2_sample.shape[1]
x3_dim = x3_sample.shape[1]
c_dim = c_sample.shape[1]

def objective(trial):
    # 하이퍼파라미터 제안
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    
    # 구조적 차원 (계층 구조 반영)
    z_dim = trial.suggest_int("z_dim", 16, 48)
    z2_dim = trial.suggest_int("z2_dim", 8, 32)
    z3_dim = trial.suggest_int("z3_dim", 4, 16)

    # 가중치 (Loss Balancing)
    a = trial.suggest_float("a", 1.0, 10.0)   # x 복원 (중요)
    b = trial.suggest_float("b", 0.1, 2.0)    # x2 복원
    c = trial.suggest_float("c", 1.0, 5.0)    # x3 복원
    d = trial.suggest_float("d", 10.0, 50.0)  # BCE 확률 예측 (가장 중요할 경우)
    
    # KL 가중치는 상대적으로 작게 관리 (정규화 강도 제어)
    e = trial.suggest_float("e", 0.1, 1.0)
    f = trial.suggest_float("f", 0.1, 1.0)
    g = trial.suggest_float("g", 0.1, 1.0)
    # 3. 모델 및 옵티마이저 선언
    model = MultiDecoderCondVAE(x_dim,x2_dim,x3_dim, c_dim, z_dim, z2_dim,z3_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 각 trial당 학습할 에포크 수 (성능 확인을 위해 최소 20~50회 권장)
    epochs = 50 
    
    for epoch in range(epochs):
        # --- Training Loop ---
        model.train()
        for x, x2,x3,c in train_loader:
            x, x2,x3, c = x.to(device),x2.to(device),x3.to(device), c.to(device)
            optimizer.zero_grad()
            
            bce_logit ,binary_out, x_hat,x2_hat,x3_hat, z_mu,z_logvar,z2_mu,z2_logvar,z3_mu,z3_logvar = model( x,x2,x3, c)
            
            # integrated_loss_fn에 제안된 가중치 적용
            loss_dict = integrated_loss_fn(
                bce_logit, x_hat, x2_hat,x3_hat,x,x2,x3, z_mu, z_logvar,z2_mu,z2_logvar,z3_mu,z3_logvar,
                a=a, b=b,c=c,d=d,e=e,f=f,g=g
            )
            
            loss_dict['loss'].backward()
            optimizer.step()
        
        # --- Validation Loop ---
        model.eval()
        v_total_loss = 0
        with torch.no_grad():
            for v_x,v2_x,v3_x, v_c in val_loader:
                v_x,v2_x,v3_x, v_c = v_x.to(device),v2_x.to(device),v3_x.to(device), v_c.to(device)
                v_bce_logit,v_binary_out,v_x_hat,v2_x_hat,v3_x_hat,v_mu,v_logvar,v2_mu,v2_logvar,v3_mu,v3_logvar = model(v_x,v2_x,v3_x, v_c)
                
                v_loss_dict = integrated_loss_fn(
                    v_bce_logit, v_x_hat,v2_x_hat,v3_x_hat, v_x,v2_x,v3_x, v_mu, v_logvar,v2_mu,v2_logvar,v3_mu,v3_logvar,a=a,b=b,c=c,d=d,e=e,f=f,g=g)
                v_total_loss += v_loss_dict['loss'].item()
        
        avg_val_loss = v_total_loss / len(val_loader)
        
        # Pruning: 성능이 개선되지 않는 trial은 조기 종료하여 시간 절약
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

# 4. 최적화 실행
# n_trials: 총 시도 횟수 (예: 30)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# 5. 최적화 결과 확인 및 모델 재학습 활용
print("-" * 30)
print("Best hyperparameters:", study.best_params)
print("Best validation loss:", study.best_value)

# 6. (선택) 파라미터 중요도 시각화
# optuna.visualization.plot_param_importances(study).show()

[I 2026-01-27 23:34:05,959] A new study created in memory with name: no-name-76587abb-dea6-4462-964f-9333fcdda2e4
[W 2026-01-27 23:34:09,612] Trial 0 failed with parameters: {'lr': 0.0004094487394712333, 'z_dim': 44, 'z2_dim': 16, 'z3_dim': 14, 'a': 6.920368713612448, 'b': 0.6651069204620611, 'c': 4.34731138293543, 'd': 44.040055452044065, 'e': 0.4214198130663106, 'f': 0.8454789531216924, 'g': 0.9166568899834714} because of the following error: RuntimeError('grad can be implicitly created only for scalar outputs').
Traceback (most recent call last):
  File "C:\Users\mingyu\AppData\Roaming\Python\Python313\site-packages\optuna\study\_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\mingyu\AppData\Local\Temp\ipykernel_13224\514476804.py", line 67, in objective
    loss_dict['loss'].backward()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\mingyu\AppData\Roaming\Python\Python313\site-packages\torch\_tensor.py", line 625, in backward
    torch.aut

RuntimeError: grad can be implicitly created only for scalar outputs